In [1]:
%matplotlib inline
%load_ext tensorboard

import matplotlib.pyplot as plt
import nengo
import numpy as np
import tensorflow as tf

import datetime, os
from itertools import islice
from IPython.display import clear_output
import pytry
import pickle
import random

import nengo_dl

C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\nengo_dl\converter.py:305: UserWarning: Layer '<class 'tensorflow.python.keras.layers.normalization_v2.BatchNormalization'>' already has a converter. Overwriting.
  "Layer '%s' already has a converter. Overwriting." % keras_layer


# Deep Learning Nengo Network

Tensorflow version 2.2.0 <br>
Nengo dl version 3.0.0

In [2]:
class FeedForwardTrain(pytry.Trial):
    def params(self):
        self.param('run_n', run_n=1)
        self.param('theta', theta=3)
        self.param('q', q=4)
        self.param('lr', lr=0.000001)
    
    def evaluate(self, param):       
        #LOAD/SET PARAMETERS
        run_n=param.run_n     
        theta=param.theta  
        q=param.q  
        learn_rate=param.lr #learning rate
        n_epoch=1000 #number of training epochs
        train_rat=0.7 #ratio of training to testing data
        
        print(run_n) #print run number
        np.random.seed() #set seed to random value
        
#################################################################
        #Set and create data directories
        data_dir="D:\\NEN002\\GridSearch_NDL_Raw\\"+str(learn_rate)+"\\" #directory to save data to
        
        if run_n == 0:
            os.mkdir(data_dir)
            os.mkdir("D:\\NEN002\\GridSearch_NDL_Raw_logs\\"+str(learn_rate)+"\\")
            
        logdir = os.path.join("D:\\NEN002\\GridSearch_NDL_Raw_logs\\"+str(learn_rate)+"\\"+str(run_n)) #directory to save logs to for tensorboard
        
        #load data
        highengdata = np.load('higheng.npy', allow_pickle=True) #np.load('high_lmu'+str(q)+'_theta'+str(theta)+'.npy',allow_pickle=True) 
        lowengdata = np.load('loweng.npy', allow_pickle=True) #np.load('low_lmu'+str(q)+'_theta'+str(theta)+'.npy',allow_pickle=True) 
##################################################################

        #high=list(highengdata[:])
        #low=list(lowengdata[:])
        
        high=list(highengdata[:,1])
        low=list(lowengdata[:,1])

        #shuffle clips
        np.random.shuffle(high)
        np.random.shuffle(low)

        #get training clips and convert to list of frames
        high_train = high[:(int(len(low)*train_rat))]
        high_train=list(np.vstack(high_train))
        low_train = low[:(int(len(low)*train_rat))]
        low_train=list(np.vstack(low_train))

        #shuffle frames
        np.random.shuffle(high_train)
        np.random.shuffle(low_train)

        #Make each training set same size (low is shortest)
        high_train = high_train[:(int(len(low_train)))]
        low_train = low_train[:(int(len(low_train)))]

        #Create 10% validation dataset
        high_val = high[(int(len(low)*0.9)):(int(len(low)*1))]
        low_val = low[(int(len(low)*0.9)):(int(len(low)*1))]
        
        #convert to list of frames
        high_val=list(np.vstack(high_val))
        low_val=list(np.vstack(low_val))

        #shuffle frames
        np.random.shuffle(high_val)
        np.random.shuffle(low_val)
        
        #create test sets
        #get 30% of clips for testing
        high_test = high[(int(len(low)*train_rat)):(int(len(low)*0.9))]
        low_test = low[(int(len(low)*train_rat)):(int(len(low)*0.9))]

        #shuffle frames
        np.random.shuffle(high_test)
        np.random.shuffle(low_test)

        #GENERATE INPUT MATRICES
        #Concatenate high and low sets together to create a single array for training, test and validation separately
        all_train = np.vstack(np.concatenate((high_train, low_train)))
        all_test = np.concatenate((high_test, low_test)) 
        all_val = np.concatenate((high_val, low_val))

        #create the target data for training
        target_train = np.zeros((all_train.shape[0],2))
        n_high = len(high_train)
        target_train[:n_high,0] = 1 #target for high = [1,0]
        target_train[n_high:,1] = 1 #target for low = [0,1]
        
        #and for validation
        target_val = np.zeros((all_val.shape[0],2))
        n_high = len(high_val)
        target_val[:n_high,0] = 1
        target_val[n_high:,1] = 1
        
        #SAVE TRAINING AND TESTING DATA
        pickle_filename = (data_dir+"/%s_training_data.pkl") % str(run_n) #(data_dir+"/%s_training_data"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(all_train, file)

        pickle_filename = (data_dir+"/%s_testing_data.pkl") % str(run_n) #(data_dir+"/%s_testing_data"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(all_test, file)

        #BUILD MODEL (input layer -> hidden layer -> output layer)
        N = 200
        seed = 1
        N_dims = len(all_train[0]) 

        model = nengo.Network(seed=seed)
        with model:    
            input = nengo.Node(np.zeros(N_dims))
            hidden1 = nengo.Ensemble(n_neurons=N, dimensions=N_dims, radius=np.sqrt(N_dims), 
                                     neuron_type=nengo.RectifiedLinear())
            nengo.Connection(input, hidden1, synapse=None)
            output = nengo.Node(None, size_in=2)
            nengo.Connection(hidden1, output, eval_points=all_train, function=target_train, 
                             scale_eval_points=False, synapse=None)

            p_output = nengo.Probe(output)
        
        #TRAINING DATA AND PARAMETERS
        minibatch_size = 1000       # this can be adjusted to speed up training
                                    # Note: with larger minibatch_size, you may need to reduce this
                                    #(if training is making things worse, then you need to reduce this!)
                                    # number of iterations through the training data to perform

        batches = int(np.ceil(len(all_train)/minibatch_size)) #number of training batches needed (length of data / minibatch size)
        val_batches = int(np.ceil(len(all_val)/minibatch_size)) #number of validation batches needed

        training_data_input = np.array(all_train, copy=True)
        order = np.arange(len(training_data_input))
        np.random.shuffle(order)
        training_data_input_rand = training_data_input[order]
        training_data_input_rand.resize(minibatch_size,batches,N_dims)
        training_data_input.resize(minibatch_size,batches,N_dims)

        training_data_output = np.array(target_train, copy=True)
        training_data_output_rand = training_data_output[order]
        training_data_output.resize(minibatch_size,batches,2)
        training_data_output_rand.resize(minibatch_size,batches,2)
        
        val_input = np.array(all_val, copy=True)
        order = np.arange(len(val_input))
        np.random.shuffle(order)
        val_input_rand = val_input[order]
        val_input_rand.resize(minibatch_size,val_batches,N_dims)
        val_input.resize(minibatch_size,val_batches,N_dims)
        
        val_output = np.array(target_val, copy=True)
        val_output_rand = val_output[order]
        val_output.resize(minibatch_size,val_batches,2)
        val_output_rand.resize(minibatch_size,val_batches,2)
        
        #set learning objective (defines loss)
        def objective(outputs, targets): 
            return tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2(
                logits=outputs, labels=targets)  
        
        #TRAINING
        with nengo_dl.Simulator(
                model, minibatch_size=minibatch_size) as sim:
            sim.compile(optimizer=tf.optimizers.Adam(learn_rate),
                loss={p_output: objective})
            sim.fit(training_data_input_rand, training_data_output_rand, epochs=n_epoch,
                   callbacks=[tf.keras.callbacks.TensorBoard(log_dir=logdir)],
                   validation_data=(val_input_rand, val_output_rand))
            sim.save_params(data_dir+"/%s_trained.data" % str(run_n)) #(data_dir+"/%s"+str(test_param)+"_trained.data") % run_n #('./trained.data') 
            
        #Run model with training data, using trained weights
        input.output = nengo.processes.PresentInput(all_train, presentation_time=0.001)
        with nengo_dl.Simulator(model, minibatch_size=1) as sim:
            sim.load_params(data_dir+"/%s_trained.data" % str(run_n)) #(data_dir+"/%s"+str(test_param)+"_trained.data") % run_n
            sim.run(len(all_train)*0.001)
            
        #Save data
        pickle_filename = (data_dir+"/%s_train_target.pkl") % str(run_n) #(data_dir+"/%s_train_target"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(target_train, file)
        
        pickle_filename = (data_dir+"/%s_train_trange.pkl") % str(run_n) #(data_dir+"/%s_train_trange"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(sim.trange(), file)
            
        pickle_filename = (data_dir+"/%s_train_out.pkl") % str(run_n) #(data_dir+"/%s_train_out"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(sim.data[p_output][0], file)    
        
        test_out=[]
        target_test=[]
        for j in range(len(all_test)):
            if j<int(len(high_test)):
                target = np.zeros((all_test[j].shape[0],2))
                target[:,0] = 1
            else:
                target = np.zeros((all_test[j].shape[0],2))
                target[:,1] = 1
            input.output = nengo.processes.PresentInput(all_test[j], presentation_time=0.001)
            with nengo_dl.Simulator(model, minibatch_size=1) as sim:
                sim.load_params(data_dir+"/%s_trained.data" % str(run_n)) #(data_dir+"/%s"+str(test_param)+"_trained.data") % run_n
                sim.run(len(all_test[j])*0.001)
            test_out.append(sim.data[p_output][0])
            target_test.append(target)
            
        #Save data
        pickle_filename = (data_dir+"/%s_test_target.pkl") % str(run_n) #(data_dir+"/%s_test_target"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(target_test, file)
        
        pickle_filename = (data_dir+"/%s_test_trange.pkl") % str(run_n) #(data_dir+"/%s_test_trange"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(sim.trange(), file)
            
        pickle_filename = (data_dir+"/%s_test_out.pkl") % str(run_n) #(data_dir+"/%s_test_out"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(test_out, file)


In [ ]:
for run in range(2,20):
#for lr in (0.001, 0.0001, 0.00001, 0.000001):
    FeedForwardTrain().run(run_n=run, lr=0.001)

running FeedForwardTrain#20200630-152322-602e7168
2
|#####################Building network (57%)                     | ETA: 0:00:00

C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\nengo_dl\simulator.py:460: UserWarning: No GPU support detected. It is recommended that you install tensorflow-gpu (`pip install tensorflow-gpu`).
  "No GPU support detected. It is recommended that you "


Build finished in 0:00:02                                                      
Optimization finished in 0:00:00                                               
Train on 1000 samples, validate on 1000 samples


Epoch 1/1000
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.8029 - probe_loss: 0.8029 - val_loss: -15.1345 - val_probe_loss: -15.1345
Epoch 2/1000
1000/1000 [==============================] - 0s 253us/sample - loss: -15.1324 - probe_loss: -15.1324 - val_loss: -31.1020 - val_probe_loss: -31.1020
Epoch 3/1000
1000/1000 [==============================] - 0s 304us/sample - loss: -31.0917 - probe_loss: -31.0917 - val_loss: -47.1015 - val_probe_loss: -47.1015
Epoch 4/1000
1000/1000 [==============================] - 0s 426us/sample - loss: -47.0828 - probe_loss: -47.0828 - val_loss: -63.1350 - val_probe_loss: -63.1350
Epoch 5/1000
1000/1000 [==============================] - 0s 283us/sample - loss: -63.1077 - probe_loss: -63.1077 - val_loss: -79.2041 - val_probe_loss: -79.2041
Epoch 6/1000
1000/1000 [==============================] - 0s 221us/sample - loss: -79.1681 - probe_loss: -79.1681 - val_loss: -95.3108 - val_probe_loss: -95.3108
Epoch 7/1000
1000/1000 [==========

1000/1000 [==============================] - 0s 227us/sample - loss: -839.4669 - probe_loss: -839.4669 - val_loss: -859.1096 - val_probe_loss: -859.1096
Epoch 51/1000
1000/1000 [==============================] - 0s 285us/sample - loss: -858.3246 - probe_loss: -858.3246 - val_loss: -878.0773 - val_probe_loss: -878.0773
Epoch 52/1000
1000/1000 [==============================] - 0s 243us/sample - loss: -877.2661 - probe_loss: -877.2661 - val_loss: -897.1311 - val_probe_loss: -897.1311
Epoch 53/1000
1000/1000 [==============================] - 0s 210us/sample - loss: -896.2922 - probe_loss: -896.2922 - val_loss: -916.2701 - val_probe_loss: -916.2701
Epoch 54/1000
1000/1000 [==============================] - 0s 234us/sample - loss: -915.4039 - probe_loss: -915.4039 - val_loss: -935.4955 - val_probe_loss: -935.4955
Epoch 55/1000
1000/1000 [==============================] - 0s 218us/sample - loss: -934.6017 - probe_loss: -934.6017 - val_loss: -954.8086 - val_probe_loss: -954.8086
Epoch 56/100

...

1000/1000 [==============================] - 0s 233us/sample - loss: -49040.0391 - probe_loss: -49040.0391 - val_loss: -49513.1445 - val_probe_loss: -49513.1445
Epoch 613/1000
1000/1000 [==============================] - 0s 231us/sample - loss: -49237.9766 - probe_loss: -49237.9766 - val_loss: -49713.0352 - val_probe_loss: -49713.0352
Epoch 614/1000
1000/1000 [==============================] - 0s 247us/sample - loss: -49436.4531 - probe_loss: -49436.4531 - val_loss: -49913.3438 - val_probe_loss: -49913.3438
Epoch 615/1000
1000/1000 [==============================] - 0s 253us/sample - loss: -49635.5234 - probe_loss: -49635.5234 - val_loss: -50114.2773 - val_probe_loss: -50114.2773
Epoch 616/1000
1000/1000 [==============================] - 0s 273us/sample - loss: -49835.1523 - probe_loss: -49835.1523 - val_loss: -50315.7031 - val_probe_loss: -50315.7031
Epoch 617/1000
1000/1000 [==============================] - 0s 255us/sample - loss: -50035.3594 - probe_loss: -50035.3594 - val_loss: -

1000/1000 [==============================] - 0s 283us/sample - loss: -58751.4414 - probe_loss: -58751.4414 - val_loss: -59314.1133 - val_probe_loss: -59314.1133
Epoch 659/1000
1000/1000 [==============================] - 0s 379us/sample - loss: -58976.4453 - probe_loss: -58976.4453 - val_loss: -59541.1758 - val_probe_loss: -59541.1758
Epoch 660/1000
1000/1000 [==============================] - 0s 277us/sample - loss: -59202.0273 - probe_loss: -59202.0273 - val_loss: -59768.6836 - val_probe_loss: -59768.6836
Epoch 661/1000
1000/1000 [==============================] - 0s 333us/sample - loss: -59428.2344 - probe_loss: -59428.2344 - val_loss: -59996.9492 - val_probe_loss: -59996.9492
Epoch 662/1000
1000/1000 [==============================] - 0s 356us/sample - loss: -59655.0469 - probe_loss: -59655.0469 - val_loss: -60225.8125 - val_probe_loss: -60225.8125
Epoch 663/1000
1000/1000 [==============================] - 0s 266us/sample - loss: -59882.4531 - probe_loss: -59882.4531 - val_loss: -

1000/1000 [==============================] - 0s 247us/sample - loss: -69788.0469 - probe_loss: -69788.0469 - val_loss: -70453.3672 - val_probe_loss: -70453.3672
Epoch 705/1000
1000/1000 [==============================] - 0s 228us/sample - loss: -70043.9062 - probe_loss: -70043.9062 - val_loss: -70711.5078 - val_probe_loss: -70711.5078
Epoch 706/1000
1000/1000 [==============================] - 0s 258us/sample - loss: -70300.3828 - probe_loss: -70300.3828 - val_loss: -70970.3750 - val_probe_loss: -70970.3750
Epoch 707/1000
1000/1000 [==============================] - 0s 240us/sample - loss: -70557.5781 - probe_loss: -70557.5781 - val_loss: -71229.8203 - val_probe_loss: -71229.8203
Epoch 708/1000
1000/1000 [==============================] - 0s 243us/sample - loss: -70815.3828 - probe_loss: -70815.3828 - val_loss: -71490.0625 - val_probe_loss: -71490.0625
Epoch 709/1000
1000/1000 [==============================] - 0s 251us/sample - loss: -71073.8594 - probe_loss: -71073.8594 - val_loss: -

1000/1000 [==============================] - 0s 327us/sample - loss: -82242.1484 - probe_loss: -82242.1484 - val_loss: -83019.4297 - val_probe_loss: -83019.4297
Epoch 751/1000
1000/1000 [==============================] - 0s 271us/sample - loss: -82528.4922 - probe_loss: -82528.4922 - val_loss: -83308.4844 - val_probe_loss: -83308.4844
Epoch 752/1000
1000/1000 [==============================] - 0s 242us/sample - loss: -82815.5391 - probe_loss: -82815.5391 - val_loss: -83597.9531 - val_probe_loss: -83597.9531
Epoch 753/1000
1000/1000 [==============================] - 0s 240us/sample - loss: -83103.2109 - probe_loss: -83103.2109 - val_loss: -83888.2344 - val_probe_loss: -83888.2344
Epoch 754/1000
1000/1000 [==============================] - 0s 250us/sample - loss: -83391.6016 - probe_loss: -83391.6016 - val_loss: -84179.2109 - val_probe_loss: -84179.2109
Epoch 755/1000
1000/1000 [==============================] - 0s 259us/sample - loss: -83680.6250 - probe_loss: -83680.6250 - val_loss: -

1000/1000 [==============================] - 0s 228us/sample - loss: -96123.7656 - probe_loss: -96123.7656 - val_loss: -97023.7109 - val_probe_loss: -97023.7109
Epoch 797/1000
1000/1000 [==============================] - 0s 234us/sample - loss: -96442.6797 - probe_loss: -96442.6797 - val_loss: -97345.5156 - val_probe_loss: -97345.5156
Epoch 798/1000
1000/1000 [==============================] - 0s 266us/sample - loss: -96762.3438 - probe_loss: -96762.3438 - val_loss: -97667.9922 - val_probe_loss: -97667.9922
Epoch 799/1000
1000/1000 [==============================] - 0s 286us/sample - loss: -97082.8516 - probe_loss: -97082.8516 - val_loss: -97991.5781 - val_probe_loss: -97991.5781
Epoch 800/1000
1000/1000 [==============================] - 0s 217us/sample - loss: -97404.1094 - probe_loss: -97404.1094 - val_loss: -98315.9453 - val_probe_loss: -98315.9453
Epoch 801/1000
1000/1000 [==============================] - 0s 255us/sample - loss: -97726.2188 - probe_loss: -97726.2188 - val_loss: -

To view logs in tensorboard use following command in command prompt: <br>
    tensorboard --logdir={logs_base_dir} --host=localhost <br>
<br>
Note: change {logs_base_dir} to the name of the folder where logs are being saved. <br>
Note: make sure you have navigated to the directory containing this script in the command prompt before running the above command.